# Ridge Regression From Scratch

First, we import numpy for the mathematical operations and pandas to load in the dataset we will test our model on.

In [8]:
import numpy as np
import pandas as pd

We create the RidgeRegression class. Alpha is the regularization rate. The following equation tells us how to calculate the weights, it is equivalent to the condition that $RSS(w)$ is minimal.

$$w_{ols} = (X^{T}X + \alpha I^{*})^{-1}Xy$$
Also, as long as $\alpha \not= 0$, we know $X^{T}X + \alpha I^{*}$ is always invertible. For any vector $x \in R^{p+1}, x \not = \bold 0$, we can see

$$x^{T}(X^{T}X + \alpha I^{*})x = x^{T}X^{T}Xx + x^{T} \alpha I^{*}x = ||Xx||^{2} + \alpha \sum^{p}_{i = 1} w^{2}_i > 0 $$

$I^{*}$ is identity matrix except with 0 in the first row and first column, we do not want to penalize the intercept.

In [9]:
class RidgeRegression:
  def __init__(self, feature_count, alfa = 0):
    self.weights = np.zeros(feature_count)
    self.alfa = alfa

  def predict(self, X_data):
    X_data = np.hstack((np.ones((X_data.shape[0], 1)), X_data))
    result = np.matmul(X_data, self.weights)
    return result
  
  def fit(self, X_data, y_data):
    X_data = np.hstack((np.ones((X_data.shape[0], 1)), X_data))

    X = np.array(X_data)
    y = np.array(y_data)

    identity_matrix = np.eye(X.shape[1])
    identity_matrix[0][0] = 0

    self.weights = np.matmul(np.linalg.inv(np.matmul(X.T, X) + self.alfa * identity_matrix), np.matmul(X.T, y))

We download dataset where the target variable is price of a house. We are not going to go in depth explaining the dataset as it is not the goal of this project. We quickly check for any missing values and convert all non-numeric values to numeric represenations.

Source: https://www.kaggle.com/datasets/yasserh/housing-prices-dataset

In [10]:
dataset = pd.read_csv("Housing.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.replace({'yes': 1, 'no': 0, 'furnished' : 1, 'semi-furnished' : 2, 'unfurnished' : 3}, inplace=True)
dataset.isna().sum()

We divide the dataset into training set and test set (80% to 20%), no need for a validation/dev set.

In [12]:
train_size = int(np.floor(0.8 * len(dataset)))

X_train = dataset.iloc[:train_size].drop(columns="price")
y_train = dataset.iloc[:train_size]["price"]
X_test = dataset.iloc[train_size:].drop(columns="price")
y_test = dataset.iloc[train_size:]["price"]

We finally try to fit our implementation with $\alpha = 1$ and then we print out the $RMSE = \sqrt{\frac{1}{N} \sum^{N}_{i=1}(Y_i - \hat{Y}_i)^{2}}$, where $N$ is the number of data points. The RMSE seems fairly high, so let's check with sklearn implementation.

In [13]:
model = RidgeRegression(dataset.shape[1] - 1, 1)
model.fit(X_train, y_train)
RMSE = np.sqrt(sum((y_test - model.predict(X_test))**2) / len(y_test))
print(f"RMSE (alfa = {model.alfa}): {round(RMSE, 3)}.")

RMSE (alfa = 1): 1111927.384.


We use the same $\alpha$ and we can see the $RMSE$ is exactly the same, therefore our implementation is correct. The is most likely not tuning $\alpha$ and not preprocessing the dataset in any way, we could definitely reach lower values of $RMSE$, but that is not the point of this project. Our implementation was successful.

In [15]:
from sklearn.linear_model import Ridge
model = Ridge(1)
model.fit(X_train, y_train)
RMSE = np.sqrt(sum((y_test - model.predict(X_test))**2) / len(y_test))
print(f"RMSE (alfa = {1}): {round(RMSE, 3)}.")

RMSE (alfa = 1): 1111927.384.
